In [1]:
from config import *

import mPyPl as mp
from mPyPl.utils.flowutils import *
from mpyplx import *
from pipe import Pipe
from functools import partial

import numpy as np
import cv2
import itertools
from moviepy.editor import *
import pickle
import functools 

In [2]:
from config import *

test_names = (
   from_json(os.path.join(source_dir,'matches.json'))
   | mp.where(lambda x: 'Test' in x.keys() and int(x['Test'])>0)
   | mp.apply(['Id','Half'],'pattern',lambda x: "{}_{}_".format(x[0],x[1]))
   | mp.select_field('pattern')
   | mp.as_list
)

In [60]:
stream = (
    mp.get_datastream(data_dir, ext=".fflow.pickle", classes={'noshot' : 0, 'shots': 1})
    | datasplit_by_pattern(test_pattern=test_names)
    | stratify_sample_tt()
    | mp.apply(['class_id','split'],'descr',lambda x: "{}-{}".format(x[0],x[1]))
    | summarize('descr')
    | mp.as_list
)

train, test = (
    stream
    | mp.apply('filename', 'raw', lambda x: pickle.load(open(x, 'rb')), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('raw', 'gradients', calc_gradients, eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('gradients', 'polar', lambda x: to_polar(x), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('polar', 'channel1', lambda x: np.concatenate([y[0] for y in x]), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('polar', 'channel2', lambda x: np.concatenate([y[1] for y in x]), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.make_train_test_split()
)

In [61]:
train = train | mp.as_list

ch1 = stream  | mp.select_field('channel1') | mp.as_list
ch1_flatten = np.concatenate(ch1)

ch2 = stream  | mp.select_field('channel2') | mp.as_list
ch2_flatten = np.concatenate(ch2)

In [72]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.hist(ch1_flatten, bins=100);

In [73]:
plt.hist(ch2_flatten, bins=100);

## OpticalFlow Model Training

In [3]:
scene_changes = pickle.load(open('scene.changes.pkl', 'rb'))
scene_changes = list(scene_changes[40].keys())
scene_changes = [ fn.replace('.resized.mp4', '.fflow.pickle') for fn in scene_changes]

In [4]:
retinaflow_shape = (25, 50, 2)

hist_params = [
    dict(
        bins=retinaflow_shape[1],
        lower=0,
        upper=150,
        maxv=150
    ),
    dict(
        bins=retinaflow_shape[1],
        lower=0,
        upper=6.29,
        maxv=6.29 
    ),
]

stream = (
    mp.get_datastream(data_dir, ext=".fflow.pickle", classes={'noshot' : 0, 'shots': 1})
    | mp.filter('filename', lambda x: not x in scene_changes) 
    | datasplit_by_pattern(test_pattern=test_names)
    | stratify_sample_tt()
    | mp.apply(['class_id','split'],'descr',lambda x: "{}-{}".format(x[0],x[1]))
    | summarize('descr')
    | mp.as_list
)

train, test = (
    stream
    | mp.apply('filename', 'raw', lambda x: pickle.load(open(x, 'rb')), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('raw', 'gradients', calc_gradients, eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('gradients', 'polar', lambda x: to_polar(x), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('polar', 'histograms', lambda x: video_to_hist(x, hist_params), eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.apply('histograms', 'fflows', functools.partial(zero_pad,shape=retinaflow_shape), 
               eval_strategy=mp.EvalStrategies.LazyMemoized)
    | mp.make_train_test_split()
)


In [5]:
no_train = stream | mp.filter('split',lambda x: x==mp.SplitType.Train) | mp.count
no_test = stream | mp.filter('split',lambda x: x==mp.SplitType.Test) | mp.count

# training params
LEARNING_RATE = 0.001
V = "v1"
MODEL_CHECKPOINT = "models/unet_ch_" + V + ".h5"
MODEL_PATH = MODEL_CHECKPOINT.replace("_ch_", "_model_")
HISTORY_PATH = MODEL_PATH.replace(".h5", "_history.pkl")
BATCH_SIZE = 16
EPOCHS = 50

In [6]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

callback_checkpoint = ModelCheckpoint(
    MODEL_CHECKPOINT, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True
)

callback_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=7, 
    verbose=1, 
    mode='auto', 
    restore_best_weights=True
)


from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.5,
                              patience=4, cooldown=4, min_lr=0.0001)

In [8]:
from keras.models import Sequential
from keras.layers import *
from keras.regularizers import l2
from keras.optimizers import Adam

retinaflow_shape = (25, 50, 2)

model = Sequential()
model.add(Conv2D(64, (5,3), input_shape=retinaflow_shape))
model.add(Conv2D(32, (3,3), activation='relu', kernel_initializer='glorot_uniform'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(32, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['acc'])
model.summary()

In [9]:
history = model.fit_generator(
        train | mp.infshuffle |  mp.as_batch('fflows', 'class_id', batchsize=BATCH_SIZE),
        steps_per_epoch = no_train // BATCH_SIZE,
        validation_data = test | mp.infshuffle | mp.as_batch('fflows', 'class_id', batchsize=BATCH_SIZE),
        validation_steps = no_test // BATCH_SIZE,
        epochs=EPOCHS, 
        verbose=1,
        callbacks=[callback_checkpoint, callback_stopping, reduce_lr]
    )

In [10]:
%matplotlib inline

import matplotlib.pyplot as plt

def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [11]:
plot_history(history)